In [18]:
import os
import math
import numpy as np
import pandas as pd
import datetime as dt
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objects as go

from itertools import cycle
from plotly.subplots import make_subplots
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, \
                            r2_score, mean_poisson_deviance, mean_gamma_deviance, accuracy_score 

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM

In [19]:
df_eth = pd.read_csv('data/ETH-USD.csv')
print('Total number of days present in the dataset: ',df_eth.shape[0])
print('Total number of fields present in the dataset: ',df_eth.shape[1])

Total number of days present in the dataset:  1806
Total number of fields present in the dataset:  7


In [20]:
df_eth.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2017-11-09,308.644989,329.451996,307.056000,320.884003,320.884003,893249984
1,2017-11-10,320.670990,324.717987,294.541992,299.252991,299.252991,885985984
2,2017-11-11,298.585999,319.453003,298.191986,314.681000,314.681000,842300992
3,2017-11-12,314.690002,319.153015,298.513000,307.907990,307.907990,1613479936
4,2017-11-13,307.024994,328.415009,307.024994,316.716003,316.716003,1041889984


In [21]:
df_eth.tail()

,Date,Open,High,Low,Close,Adj Close,Volume
1801,2022-10-15,1297.305908,1300.745972,1268.468506,1274.871704,1274.871704,6798512624
1802,2022-10-16,1275.005493,1312.634521,1275.005493,1306.296631,1306.296631,7491625206
1803,2022-10-17,1306.309570,1335.647827,1297.447266,1331.713623,1331.713623,9401189650
1804,2022-10-18,1331.669922,1339.085938,1291.660645,1310.447021,1310.447021,10416747806
1805,2022-10-19,1311.808594,1312.430908,1294.931396,1297.278198,1297.278198,9011793920


In [22]:
df_eth.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1806 entries, 0 to 1805
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       1806 non-null   object 
 1   Open       1806 non-null   float64
 2   High       1806 non-null   float64
 3   Low        1806 non-null   float64
 4   Close      1806 non-null   float64
 5   Adj Close  1806 non-null   float64
 6   Volume     1806 non-null   int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 98.9+ KB


In [23]:
df_eth.describe()

,Open,High,Low,Close,Adj Close,Volume
count,1806.000000,1806.000000,1806.000000,1806.000000,1806.000000,1.806000e+03
mean,1122.796847,1159.416704,1080.993382,1123.101713,1123.101713,1.298508e+10
std,1211.526394,1248.588873,1168.545708,1210.892351,1210.892351,1.087333e+10
min,84.279694,85.342743,82.829887,84.308296,84.308296,6.217330e+08
25%,207.929550,212.716999,202.549011,207.745041,207.745041,4.267517e+09
50%,466.839004,479.886871,453.149048,466.781509,466.781509,1.079156e+10
75%,1811.635559,1847.889374,1745.897278,1812.483735,1812.483735,1.829218e+10
max,4810.071289,4891.704590,4718.039063,4812.087402,4812.087402,8.448291e+10


In [24]:
print('Null Values:',df_eth.isnull().values.sum())
print('NA values:',df_eth.isnull().values.any())

Null Values: 0
NA values: False


In [25]:
df_eth.drop(df_eth[['Adj Close','Volume']],axis=1)

,Date,Open,High,Low,Close
0,2017-11-09,308.644989,329.451996,307.056000,320.884003
1,2017-11-10,320.670990,324.717987,294.541992,299.252991
2,2017-11-11,298.585999,319.453003,298.191986,314.681000
3,2017-11-12,314.690002,319.153015,298.513000,307.907990
4,2017-11-13,307.024994,328.415009,307.024994,316.716003
...,...,...,...,...,...
1801,2022-10-15,1297.305908,1300.745972,1268.468506,1274.871704
1802,2022-10-16,1275.005493,1312.634521,1275.005493,1306.296631
1803,2022-10-17,1306.309570,1335.647827,1297.447266,1331.713623
1804,2022-10-18,1331.669922,1339.085938,1291.660645,1310.447021


In [26]:
df_eth['Date'] = pd.to_datetime(df_eth['Date'])

In [27]:
monthvise = df_eth.groupby(df_eth['Date'].dt.strftime('%B'))[['Open','Close']].mean()
new_order = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 
             'September', 'October', 'November', 'December']
monthvise = monthvise.reindex(new_order, axis=0)
monthvise

,Open,Close
Date,,
January,1129.867529,1129.190249
February,1153.856283,1154.872601
March,1105.274615,1107.333834
April,1246.177901,1250.662702
May,1292.438870,1287.031380
June,959.265114,950.002085
July,886.457313,892.302205
August,1140.152248,1144.327363
September,1125.387235,1120.621615


In [28]:
names = cycle(['Stock Open Price','Stock Close Price','Stock High Price','Stock Low Price'])

fig = px.line(df_eth, x=df_eth.Date, y=[df_eth['Open'], df_eth['Close'], 
                                          df_eth['High'], df_eth['Low']],
             labels={'Date': 'Date','value':'Stock value'})
fig.update_layout(title_text='Stock analysis chart', font_size=15, font_color='black',legend_title_text='Stock Parameters')
fig.for_each_trace(lambda t:  t.update(name = next(names)))
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)

fig.show()

In [29]:
df_eth = df_eth[['Date','Close']]
print("Shape of close dataframe:", df_eth.shape)

Shape of close dataframe: (1806, 2)


In [30]:
fig = px.line(df_eth, x=df_eth.Date, y=df_eth.Close,labels={'date':'Date','close':'Close Stock'})
fig.update_traces(marker_line_width=2, opacity=0.8, marker_line_color='orange')
fig.update_layout(
                  title_text='Whole period of timeframe of Bitcoin close price 2014-2022', 
                  plot_bgcolor='white', 
                  font_color='black', 
                  font_size=15
                  )
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

In [31]:
df_eth = df_eth[df_eth['Date'] > '2021-01-01']
eth_stock = df_eth.copy()
print("Total data for prediction: ",df_eth.shape[0])

Total data for prediction:  656


In [32]:
fig = px.line(df_eth, x=df_eth.Date, y=df_eth.Close,labels={'date':'Date','close':'Close Stock'})
fig.update_traces(marker_line_width=2, opacity=0.8, marker_line_color='orange')
fig.update_layout(title_text='Considered period to predict Bitcoin close price', 
                  plot_bgcolor='white', font_size=15, font_color='black')
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()